# CorALS - Documentation

## Prepare parallelization

Before running anything, we make sure that `numpy` will not  oversubscribe CPUs and slow things down.
Note that this has to be executed **before importing `numpy`**.

* For full correlation matrix calculation, setting `n_threads > 1` can be used to parallelize the calculation.
* For the top-k approaches, setting `n_threads=1` makes the most sense, since parallelization is specified separately.

In [ ]:
from corals.threads import set_threads_for_external_libraries
set_threads_for_external_libraries(n_threads=1)

## Load data

In [ ]:
import numpy as np

# create random data
n_features = 20000
n_samples = 50
X = np.random.random((n_samples, n_features))

## Full correlation matrix computation

In [ ]:
# runtime: ~2 sec
from corals.correlation.full.default import cor_full
cor_values = cor_full(X)

### Sanity Check

Also see: `tests/test_correlation_full.py`

In [ ]:
np.random.seed(42)

# data
XX = np.random.random((5, 10))

# reference values
cor_default_values = np.corrcoef(XX, rowvar=False)

# CorALS
cor_test_values = cor_full(XX)

assert np.allclose(cor_default_values, cor_test_values)

## Top-k correlation matrix computation using Spearman correlation

In [ ]:
# runtime: ~5 sec with `n_jobs=8`
from corals.correlation.topk.default import cor_topk
cor_topk_result = cor_topk(X, k=0.001, correlation_type="spearman", n_jobs=8)

### Sanity check

Also see: `test_correlation_topk.py`

In [ ]:
np.random.seed(42)
k = 15

# data
XX = np.random.random((5, 10))

# ref
cor = np.corrcoef(XX, rowvar=False).flatten()
topk_order = np.argsort(-np.abs(cor))[:k]
topk_values = cor[topk_order]
val_ref, idx_ref = topk_values, np.unravel_index(topk_order, (X.shape[1], X.shape[1]))

# CorALS
val, idx = cor_topk(XX, k=k)

# check
assert np.allclose(val, val_ref)

## Top-k differential correlation matrix computation using Spearman correlation

In [ ]:
# generate some more data
X1 = X
X2 = np.random.random((n_samples, n_features))

In [ ]:
from corals.correlation.topkdiff.default import cor_topkdiff
cor_topkdiff_result = cor_topkdiff(X1, X2, k=0.001, correlation_type="spearman", n_jobs=8)

### Sanity check

Also see: `tests/test_correlation_topkdiff.py`

In [ ]:
np.random.seed(42)
m,n = 20, 10
k = 15

# data
m1 = (np.random.random((m,n)) - 0.5) * 10
m2 = (np.random.random((m,n)) - 0.5) * 10

# reference values
cm1 = np.corrcoef(m1, rowvar=False)
cm2 = np.corrcoef(m2, rowvar=False)
m_diff = cm1 - cm2
v_diff = m_diff.flatten()
order = np.argsort(-np.abs(v_diff))

# CorALS
topkdiff, _ = cor_topkdiff(m1, m2, k=k, correlation_type="pearson", n_jobs=8)

# check
assert np.allclose(np.abs(v_diff[order])[:k], np.abs(topkdiff))

## Calculating p-values

In [ ]:
# reusing correlation from the top-k example
# runtime: ~5 sec with `n_jobs=8`
from corals.correlation.topk.default import cor_topk
cor_topk_values, cor_topk_coo = cor_topk(X, correlation_type="spearman", k=0.001, n_jobs=8)

from corals.correlation.utils import derive_pvalues, multiple_test_correction
n_samples = X.shape[0]
n_features = X.shape[1]

# calculate p-values
pvalues = derive_pvalues(cor_topk_values, n_samples)

# multiple hypothesis correction
pvalues_corrected = multiple_test_correction(pvalues, n_features, method="fdr_bh")

### Sanity check

Also see: `tests/test_utils.py`.

In [ ]:
import scipy.stats

# data
XX = np.random.random((10, 5))

# reference values
correlations = []
pvalues = []
for i in range(XX.shape[1]):
    for j in range(XX.shape[1]):
        r, p = scipy.stats.pearsonr(XX[:,i], XX[:,j])
        correlations.append(r)
        pvalues.append(p)
correlations = np.asarray(correlations)
pvalues = np.asarray(pvalues)

# CorALS
derived_pvalues = derive_pvalues(correlations, XX.shape[0])

# check
assert np.all(np.isclose(pvalues, derived_pvalues))

In [ ]:
# NOTE: requires `statsmodels` which is not part of the required packages of CorALS
import statsmodels.stats.multitest
rnd = np.random.RandomState(42)

# data
pvalues = rnd.random(4 * 4) / 10

# data
_, pvalues_bonferroni, _, _ = statsmodels.stats.multitest.multipletests(pvalues, method="bonferroni")
_, pvalues_fdr_bh, _, _ = statsmodels.stats.multitest.multipletests(pvalues, method="fdr_bh")

# CorALS
pvalues_bonferroni_test = multiple_test_correction(pvalues.flatten(), 4, "bonferroni", minimal_pvalues=False)
pvalues_fdr_bh_test = multiple_test_correction(pvalues.flatten(), 4, "fdr_bh", minimal_pvalues=False)

# check
assert np.array_equiv(pvalues_bonferroni, pvalues_bonferroni_test)
assert np.array_equiv(pvalues_fdr_bh, pvalues_fdr_bh_test)


## Recommendations

### Full correlation matrix calculation

CorALS generally outperforms comparable full correlation matrix methods like `numpy.corrcoef`.
Thus, we generally recommend using *CorALS* for full correlation matrix estimation as long as the final matrix fits into memory.
Otherwise, top-k estimation may be a better choice.

### Top-k correlation discovery

For top-k correlation search, we recommend using the basic CorALS implementation (referred to as matrix in Table 3) as long as the full correlation matrix fits into memory, independent of the number of samples.

However, as the number of features increases, memory issues will make this approach impossible to use. When this is the case, switching to the index based CorALS implementation is the best option.

**Note 1**: With increasing sample numbers, CorALS becomes slower, which may warrant other heuristics such as dimensionality reduction such as locality sensitive hashing or random projections. However, this exploration is left for future work.

**Note2**:  Note that, by default, the top-k approximation approach does not guarantee symmetric results, i.e., even if `cor(x, y)` is returned, `cor(y, x)` may be missing. This can be addressed by various post-processing steps, e.g., by adding missing values. CorALS provides the option to enable this feature.